Соломин Дмитрий, гр. 932328

Лабораторная работа №2. Бинарный классификатор.
Цель работы – написать программу на языке Python, используя Jupyter Notebook, выполняющую построение и обучение нейронных сетей прямого распространения, решающих задачу бинарной классификации по вариантам, требуется подобрать безизбыточную архитектуру сети, работающей с допустимым уровнем ошибки и визуализировать процесс обучения моделей. Результаты работы привести в отчете.
В рамках данной лабораторной работы предполагается построение бинарного
классификатора с помощью полносвязной нейронной сети
Задание
1. Выберите предметную область
2. Спроектируйте полносвязную нейронную сеть с использованием Python и
TensorFlow

Классификация типов автомобилей.

1. Установка всех необходимых библиотек и импорт их в проект.

In [1]:
!pip install protobuf==3.20.3 tensorflow-intel==2.12.0
!pip install numpy pandas matplotlib seaborn
!pip install scikit-learn
!pip install tensorflow-datasets

  Using cached protobuf-3.20.3-cp38-cp38-win_amd64.whl.metadata (699 bytes)
  Using cached tensorflow_intel-2.12.0-cp38-cp38-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp38-cp38-win_amd64.whl.metadata (2.5 kB)
  Using cached jax-0.4.13.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl

In [2]:
import numpy as np  # Работа с массивами данных
import pandas as pd  # Работа с табличными данными
import matplotlib.pyplot as plt  # Построение графиков
import seaborn as sns  # Визуализация данных
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Стандартизация данных и кодирование меток классов
from sklearn.model_selection import train_test_split  # Разделение данных на обучающую и тестовую выборки
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score  # Метрики для оценки модели
from tensorflow import keras  # Импорт основной библиотеки для работы с нейронными сетями
from tensorflow.keras import layers  # Импорт слоев для построения нейронной сети
import tensorflow_datasets as tfds  # Импортируем библиотеку для загрузки набора данных Cars196

c:\Users\Dmitry\Desktop\Нейронные сети\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Загружаем набор данных https://www.tensorflow.org/datasets/catalog/cars196?hl=ru с помощью tensorflow_datasets, который содержит изображения и метки автомобилей. 
Данные автоматически разделяются на обучающие и тестовые выборки.

In [10]:
# Загрузка набора данных Cars196
data, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
train_data, test_data = data['train'], data['test']  # Разделение на обучающие и тестовые данные

Dl Completed...: 0 url [00:00, ? url/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]WARNING:absl:1738 images were corrupted and were skipped


Dataset cats_vs_dogs downloaded and prepared to C:\Users\Dmitry\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


KeyError: 'test'

3. функция, которая фильтрует только два класса для задачи бинарной классификации. В данном случае это классы 0 и 1.

In [9]:
# Фильтрация двух классов для бинарной классификации
def filter_two_classes(image, label):
    return tf.logical_or(tf.equal(label, 0), tf.equal(label, 1))  # Оставляем только классы 0 и 1

train_data = train_data.filter(filter_two_classes)  # Фильтрация обучающих данных
test_data = test_data.filter(filter_two_classes)  # Фильтрация тестовых данных


NameError: in user code:

    File "C:\Users\Dmitry\AppData\Local\Temp\ipykernel_1528\3368866555.py", line 3, in filter_two_classes  *
        return tf.logical_or(tf.equal(label, 0), tf.equal(label, 1))  # Оставляем только классы 0 и 1

    NameError: name 'tf' is not defined


4. функция которая преобразует метки классов в бинарные значения: 0 остается 0, а остальные метки преобразуются в 1.

In [ ]:
# Преобразование меток классов в бинарные значения
def relabel(label):
    return tf.where(label == 0, 0, 1)  # Присваиваем классу 0 значение 0, остальным – 1

train_data = train_data.map(lambda image, label: (image, relabel(label)))  # Преобразование меток в обучающих данных
test_data = test_data.map(lambda image, label: (image, relabel(label)))  # Преобразование меток в тестовых данных

5. Функция preprocess_image изменяет размер изображений на 128x128 пикселей и нормализует значения пикселей, чтобы ускорить обучение сети.

In [ ]:
# Изменение размера изображений и их нормализация
def preprocess_image(image, label):
    image = tf.image.resize(image, (128, 128))  # Приведение изображений к одному размеру 128x128 пикселей
    image = image / 255.0  # Нормализация изображений (приведение значений пикселей к диапазону от 0 до 1)
    return image, label

train_data = train_data.map(preprocess_image)  # Применение преобразований к обучающим данным
test_data = test_data.map(preprocess_image)  # Применение преобразований к тестовым данным


6. Разделяем данные на батчи (пакеты) по 32 изображения, чтобы ускорить обучение сети, 
так как модель будет обучаться на батчах, а не на всех данных сразу.

In [ ]:
# Создание батчей данных для эффективного обучения
train_data = train_data.batch(32)  # Разделение обучающих данных на батчи по 32 изображения
test_data = test_data.batch(32)  # Разделение тестовых данных на батчи по 32 изображения

7. Создаем нейронную сеть с тремя слоями:

- Flatten превращает изображение в одномерный вектор, чтобы его можно было передать в полносвязные слои.
- Два полносвязных слоя с 16 нейронами и функцией активации ReLU (Rectified Linear Unit), которая помогает модели справляться с нелинейностями.
- Выходной слой с функцией sigmoid, которая возвращает вероятность принадлежности к одному из двух классов (бинарная классификация).

In [ ]:
# Создание модели нейронной сети
model = keras.Sequential([
    layers.Flatten(input_shape=(128, 128, 3)),  # Преобразование входных данных в одномерный вектор
    layers.Dense(16, activation="relu"),  # Полносвязный слой с 16 нейронами и функцией активации ReLU
    layers.Dense(16, activation="relu"),  # Еще один полносвязный слой с 16 нейронами и активацией ReLU
    layers.Dense(1, activation="sigmoid")  # Выходной слой с 1 нейроном и функцией активации сигмоид для бинарной классификации
])


8. Компилируем модель, указывая оптимизатор Adam для оптимизации весов модели, функцию потерь binary_crossentropy, которая подходит для бинарной классификации, и метрику accuracy для оценки точности модели.

In [ ]:
# Компиляция модели
model.compile(optimizer="adam",  # Оптимизатор Adam для корректировки весов
              loss="binary_crossentropy",  # Функция потерь для бинарной классификации
              metrics=["accuracy"])  # Метрика для оценки точности модели
model.summary()  # Вывод структуры модели


9. Обучение модели на данных в течение 20 эпох. В процессе обучения модель также проверяет свою точность на тестовых данных.

In [ ]:
# Обучение модели
history = model.fit(train_data,  # Обучающие данные
                    epochs=20,  # Количество эпох
                    validation_data=test_data)  # Валидация на тестовых данных


10. Отображение графика потерь на обучающей и валидационной выборках, что позволяет визуализировать, как меняется ошибка модели в процессе обучения. 

In [ ]:
# Визуализация потерь на обучающих и тестовых данных
loss = history.history['loss']  # Потери на обучающей выборке
val_loss = history.history['val_loss']  # Потери на валидационной выборке
epochs = range(1, len(loss) + 1)  # Номера эпох

# Построение графика потерь
plt.plot(epochs, loss, 'y', label='Training loss')  # Потери на обучающей выборке
plt.plot(epochs, val_loss, 'r', label='Validation loss')  # Потери на валидационной выборке
plt.title('Training and validation loss')  # Заголовок графика
plt.xlabel('Epochs')  # Подпись оси X
plt.ylabel('Loss')  # Подпись оси Y
plt.legend()  # Отображение легенды
plt.show()  # Вывод графика

11. Выполняем прогнозирование на тестовых данных и вычисляем метрики точности, AUC (площадь под ROC-кривой), матрицу ошибок и отчет классификации для анализа производительности модели.

In [ ]:
# Прогнозирование на тестовой выборке
y_pred_prob = model.predict(test_data)  # Прогнозирование вероятностей
y_pred = (y_pred_prob > 0.5).astype(int)  # Преобразование вероятностей в бинарные метки (0 или 1)

# Получение истинных меток
y_test = np.concatenate([y for _, y in test_data], axis=0)  # Извлечение истинных меток из тестовой выборки

# Оценка модели
def compute_evaluation_metric(model, y_test, y_pred, y_pred_prob):
    print("\nAccuracy Score: \n", accuracy_score(y_test, y_pred))  # Точность модели
    print("\nAUC Score: \n", roc_auc_score(y_test, y_pred_prob))  # AUC (Area Under Curve) показатель
    print("\nConfusion Matrix: \n", confusion_matrix(y_test, y_pred))  # Матрица ошибок
    print("\nClassification Report: \n", classification_report(y_test, y_pred))  # Подробный отчет о точности, полноте и F1-score

compute_evaluation_metric(model, y_test, y_pred, y_pred_prob)  # Вызов функции для оценки модели


In [ ]:
print(8)